<a href="https://colab.research.google.com/github/lsh3163/Imagenet-Better/blob/master/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Do Better ImageNet Models Transfer Better?


## Library

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

## Parameter

In [0]:
NUM_GPUS = 4
BS_PER_GPU = 64 # Batchsize = 4x64
NUM_EPOCHS = 200
TASK=2 # Task 1 : Logistic Regression, Task2 : Transfer learning, Task3 : Random Initialization(Scratch Training)
MODEL = "mobilenet_v2" # mobilenet_v1, mobilenet_v2
NUM_CLASSES = 10
DATASET = "cifar10" # food101, cifar10, cifar100, sun397, oxford_flowers102, caltech101
learning_rate = 0.01

In [0]:
metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top_K")]
if DATASET=="cifar10":
  NUM_CLASSES=10
elif DATASET=="cifar100":
  NUM_CLASSES=100
elif DATASET=="food101":
  NUM_CLASSES=101
elif DATASET=="sun397":
  NUM_CLASSES=397
elif DATASET=="oxford_flowers102" or DATASET=="caltech101":
  NUM_CLASSES=102
  metrics=["acc"]

## Dataset

In [0]:
def normalize_img(image, label):
  # image = tf.image.resize(image, (224, 224))
  return tf.cast(image, tf.float32) / 127.5 - 1., label

In [5]:
(train_dataset, test_dataset), ds_info = tfds.load(
    DATASET,
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_train = train_dataset.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BS_PER_GPU * NUM_GPUS)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = test_dataset.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BS_PER_GPU * NUM_GPUS)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompletePCKJZ8/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompletePCKJZ8/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


## Model

In [6]:
if TASK==1:
    include_top=False
    weights="imagenet"
    trainable=False
elif TASK==2:
    include_top=False
    weights="imagenet"
    trainable=True
elif TASK==3:
    include_top=False
    weights=None
    trainable=True


if MODEL=="mobilenet_v1":
    base_model = tf.keras.applications.MobileNet(input_shape=None,
                                            include_top=include_top,
                                            weights=weights)
elif MODEL=="mobilenet_v2":
    base_model = tf.keras.applications.MobileNetV2(input_shape=None,
                                                 include_top=include_top,
                                                 weights=weights)

# Freeze the pre-trained model weights
base_model.trainable = trainable

# Trainable classification head
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')

# Layer classification head with feature detector
model = tf.keras.Sequential([
    base_model,
    maxpool_layer,
    prediction_layer
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(lr=learning_rate, momentum=0.9, decay=1e-6, nesterov=True),
              loss='sparse_categorical_crossentropy',
              metrics=metrics
              # metrics=["acc"]
)
print(model.summary())

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, None, None, 1280)  2257984   
_________________________________________________________________
global_max_pooling2d (Global (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 2,270,794
Trainable params: 2,236,682
Non-trainable params: 34,112
_________________________________________________________________
None


In [0]:

history = model.fit(ds_train,
          validation_data=ds_test,
          validation_freq=1,
          epochs=NUM_EPOCHS, shuffle=True)

Epoch 1/200
196/196 [==============================] - 9s 45ms/step - loss: 1.7667 - top_K: 0.3639 - val_loss: 3.2334 - val_top_K: 0.1000
Epoch 2/200
196/196 [==============================] - 6s 33ms/step - loss: 1.3527 - top_K: 0.5137 - val_loss: 2.3286 - val_top_K: 0.1447
Epoch 3/200
196/196 [==============================] - 7s 33ms/step - loss: 1.2667 - top_K: 0.5489 - val_loss: 2.1701 - val_top_K: 0.1814
Epoch 4/200
196/196 [==============================] - 7s 33ms/step - loss: 1.1087 - top_K: 0.6043 - val_loss: 2.1220 - val_top_K: 0.2236
Epoch 5/200
196/196 [==============================] - 7s 34ms/step - loss: 1.0118 - top_K: 0.6364 - val_loss: 2.0104 - val_top_K: 0.2790
Epoch 6/200
196/196 [==============================] - 7s 34ms/step - loss: 0.9188 - top_K: 0.6710 - val_loss: 1.9715 - val_top_K: 0.3126
Epoch 7/200
196/196 [==============================] - 7s 33ms/step - loss: 0.8477 - top_K: 0.6975 - val_loss: 1.8525 - val_top_K: 0.3862
Epoch 8/200
196/196 [=============